In [1]:
import pdb
import json
import pickle
import argparse
import pandas as pd
import networkx as nx
from collections import Counter
# import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
# sns.set(font_scale=1.5)
import matplotlib.pyplot as plt
from datetime import datetime, timezone
from google.colab import drive
drive.mount('/content/drive')

# 安装gensim: 
# !pip install gensim
# from gensim.models import KeyedVectors
# FILE="/content/drive/MyDrive/code/bert_related/wiki-news-300d-1M.vec"
# FASTTEXT=KeyedVectors.load_word2vec_format(FILE,limit=500000)

# 安装sentence-transformers:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

# 加载roberta-large-nli-stsb-mean-tokens
# model = SentenceTransformer("araphrase-MiniLM-L6-v2")
sentence_model = SentenceTransformer("roberta-large-nli-stsb-mean-tokens")

# 安装huggingface transformers（版本大于4.3.0）:
# !pip install transformers
# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased") 
# model = AutoModel.from_pretrained("bert-base-cased")




Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 5.8 MB 31.2 MB/s 
     |████████████████████████████████| 1.3 MB 61.0 MB/s 
     |████████████████████████████████| 182 kB 63.0 MB/s 
     |████████████████████████████████| 7.6 MB 76.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f2982c6a12b1c944c6ad5404ac92ee4b8f3b7bf917a3f9cba4f11d71e637c790
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


Downloading:   0%|          | 0.00/748 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [2]:
data = pd.read_csv("/content/drive/MyDrive/code/bert_related/labeled_data.csv", index_col = False)
data = data[data['agreement_fraction']==1.000000]
bb = data[["author_parent","author_child",'label','datetime','subreddit','body_parent', 'body_child']]
bb_sub = data[['body_parent', 'body_child']]

In [3]:
len(bb_sub)

16723

In [4]:
par_text_list = list(bb_sub['body_parent'])
chi_text_list = list(bb_sub['body_child'])

In [5]:
# par_text_list = list(bb_sub['body_parent'])
chi_text_list = list(bb_sub['body_child'])

In [ ]:
# bert model for child
import numpy as np
chi_output = []
chi_output0 = sentence_model.encode(chi_text_list[0])

for i in range(0,len(chi_text_list)):
  print(i)
  per_chi_out = sentence_model.encode(chi_text_list[i])
  chi_output.append(per_chi_out)



In [47]:
chi_output0.shape
aa = pd.DataFrame(chi_output)
print(aa)
aa.to_csv('/content/drive/MyDrive/code/bert_related/sentence_child.csv')
# bb = pd.read_csv('sentence_ber_par.csv',index_col=[0])

           0         1         2         3         4         5         6     \
0     -0.782754 -0.916833  0.283499  0.572817 -0.984099  1.168669 -1.331215   
1     -0.446615  0.121781 -0.013008  0.074826 -0.915978 -0.096690 -0.340230   
2      0.604510 -0.999536 -0.384721  0.134710 -1.010273  0.127657 -0.722045   
3     -0.134631  0.145389  0.035348  0.910113  0.387942 -0.569983 -0.146851   
4      0.154050 -0.018830  0.528967  0.379970 -0.001339  0.602786 -1.548274   
...         ...       ...       ...       ...       ...       ...       ...   
16718 -0.362124  0.076284 -0.223626  0.069595 -0.612849 -0.910001  0.441066   
16719  0.316474 -0.454696 -1.230638 -0.008043 -1.004911  0.751162  0.165707   
16720 -2.085003  0.338373  0.486263  0.214255  0.429286  0.080770  0.744443   
16721 -1.023360  0.216854 -0.445231 -0.444458 -0.239850 -0.171843 -0.885892   
16722 -0.350506 -0.165071 -0.068839  0.522192 -0.835556  0.039285  0.121176   

           7         8         9     ...      1014 

In [ ]:
bert_par['bert_par'] = np.array(bert_par.to_numpy()).tolist()
bert_chi['bert_chi'] = np.array(bert_chi.to_numpy()).tolist()
aa = pd.concat([bert_par['bert_par'], bert_chi['bert_chi']], axis=1)

aaa = aa.reset_index()    
bbb = bb.reset_index() 

cc = pd.concat([bbb, aaa], axis=1)